<a href="https://colab.research.google.com/github/tejaswishetty17/GenAI_Work/blob/main/langchain_demo9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Runnable Prallel

In [1]:
pip install -q langchain-openai langchain-core langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.8 MB/s eta 0:00:00


In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableSequence, RunnableParallel, RunnableBranch
import os

In [3]:
os.environ["OPENAI_API_KEY"] = "<API_KEY>"


In [6]:
chat_model = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=.7
)

In [7]:
def analyze_sentiment(text:str) -> str:
  """Perform sentiment analysis on the input text."""
  prompt = f"Analyze the sentiment of the following text:{text}"
  return chat_model.invoke(prompt).content

def extract_entities(text:str)-> str:
  """Extract named entities (like names, places and organization) from text."""
  prompt = f"Extract named entities from this text:{text}"
  return chat_model.invoke(prompt).content

def extract_keywords(text:str)-> str:
  """Extract import keywords from the given text."""
  prompt = f"Extract import keywords from this text: {text}"
  return chat_model.invoke(prompt).content

In [8]:
parallel_tasks = RunnableParallel(
    sentiment = RunnableLambda(analyze_sentiment),
    entities = RunnableLambda(extract_entities),
    keywords = RunnableLambda(extract_keywords)
)

In [9]:
#Formatting function to structure the output
format_results = RunnableLambda(lambda results:f"""
📊**Sentiment Analysis**:{results['sentiment']}
🔎**Named Entities**:{results['entities']}
🔑**Keywords**:{results['keywords']}
""")

In [10]:
chain = parallel_tasks | format_results

In [11]:
input_text = "Apple is planning to open a new office in San Francisco, creating many new job opportunities."

In [14]:
response = chain.invoke(input_text)
print(response)


📊**Sentiment Analysis**:The sentiment of the text is positive. It highlights a favorable development—Apple opening a new office—which is expected to create many new job opportunities. This suggests growth and economic improvement, which generally evokes positive feelings.
🔎**Named Entities**:Named entities extracted from the text:

1. Apple (Organization)
2. San Francisco (Location)
🔑**Keywords**:- Apple
- new office
- San Francisco
- job opportunities        



#Example Use-case : Prompt Templates | Chaining

In [17]:
classification_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content = """You are an AI that classifies text into 'question', 'story', 'statement'.
    only return one of these three words."""),
    ("human", "Classify the following text:\n\n{text} \n Category:")
])

In [19]:
classification_step = classification_prompt | chat_model | (lambda x: {"category": x.content.strip().lower()})

In [24]:
answer_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content = """You are an AI answers question concisely."""),
    ("human", "Answer this question:{text}")
])

In [25]:
summary_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content = """You are an AI that answers summarizes stories concisely."""),
    ("human", "Summarize this story:{text}")
])

In [26]:
title_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content = """You are an AI that generates creative and engaging titles for statements."""),
    ("human", "create a short, engaging title for the following statement:{text}")
])

In [27]:
answer_question = answer_prompt | chat_model | (lambda x: {"response": x.content.strip()})

In [28]:
summarize_story = summary_prompt | chat_model | (lambda x: {"response": x.content.strip()})

In [29]:
generate_title = title_prompt | chat_model | (lambda x: {"response": x.content.strip()})

In [33]:
pipeline = (
    RunnableLambda(lambda x: {"text" : x}) |
    (lambda x : {**x, **classification_step.invoke(x)}) |
    RunnableBranch(
        (lambda x: x["category"] == "question", answer_question),
        (lambda x: x["category"] == "story", summarize_story),
        (generate_title)
    )
)

In [15]:
inputs = [
    "What is the capital of Japan?",
    "Once upon a time in a faraway land, there was a brave knight who fought dragons to protect his kingdom.",
    "Technology is evolving rapidly, and AI is shaping the future of work.",
    "Self-driving cars will change the way we travel.",
]

In [34]:
for input_text in inputs:
  response = pipeline.invoke(input_text)
  print(f"🖊️ **Input** : {input_text} \n 🔊 **Responses {response['response']}\n {'-'*50}")


🖊️ **Input** : What is the capital of Japan? 
 🔊 **Responses The capital of Japan is Tokyo.
 --------------------------------------------------
🖊️ **Input** : Once upon a time in a faraway land, there was a brave knight who fought dragons to protect his kingdom. 
 🔊 **Responses In a distant land, a courageous knight battled dragons to safeguard his kingdom.
 --------------------------------------------------
🖊️ **Input** : Technology is evolving rapidly, and AI is shaping the future of work. 
 🔊 **Responses "AI: The Catalyst for Tomorrow's Workplace Revolution"
 --------------------------------------------------
🖊️ **Input** : Self-driving cars will change the way we travel. 
 🔊 **Responses "Revolution on Wheels: The Future of Travel with Self-Driving Cars"
 --------------------------------------------------
